<a href="https://colab.research.google.com/github/codestates/ds-cs-section1-sprint2/blob/master/n122-chi-square-test/n122a-chi-square-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 2*

# 📝 Assignment


## 1. 공공데이터 조작 연습. 

국가 통계 포털에서 아래 이미지를 참조하여

**2020년 8월**에 해당하는 규모별 미분양현황 에 대한 데이터셋을 생성하세요. 

<img src='https://i.imgur.com/aZc4UJO.png' width = 500>

- `60m이하`, `60~85m`, `85m초과`의 3개 규모와
- `서울` `대전` `대구` `부산` 4개의 지역을 포함해야합니다.
- 데이터가 없는 경우는 0으로 처리하세요.

이후 데이터셋을 colab으로 불러오세요. (방식은 상관 없습니다.)

## 2. 지역에 대해서 one-sample chi-square test를 실행하고, 결과를 설명해보세요.

9월달 데이터를 기준으로 한다면
```python
 [0+0+0+52+2+0, 0+0+0+590+665+142, 0+0+0+113+1061+42, 0+0+0+772+1+0]
```
을 비교해야할겁니다.
 


## 3. 지역과 규모에 대해서 two-sample chi-square test를 시행하고 결과를 설명해보세요.

9월달 데이터를 기준으로 한다면

| |-60 | 60-85 | 85- |
|:-:|:-:|:-:|:-:|
|서울|52|2|0|
|대전|772|1|0|
|대구|113|1061|42|
|부산|590|665|142|

에 대해서 검정해야 할 겁니다.




### 4. 2번에 대해서 NumPy 를 사용하여 (Scipy를 사용하지 않고) $\chi^2$ test 시행 후 2번의 결과와 비교해보세요. 

이 후 `chi2_contingency()`의 결과물과 비교해보세요

## 🔥 도전 과제

아래 두가지를 다하면 됩니다.

### 1. Function

4번에서 사용한 one sample chisquare test를 함수의 형태로 변경하세요.

In [ ]:
v1 = [18,22,20,15,23,22]
v2 = [5,23,26,19,24,23]
def myChisq(value):

# 내용을 채우세요.

myChisq(v1) 

# chisquare값과 pvalue를 출력해야합니다.

myChisq(v2)

### 2. ANOVA

아래 링크를 참조하여 ANOVA 에 대한 글을 읽고

<https://partrita.github.io/posts/ANOVA-python/>

다음 `4개 그룹에 대해서 평균의 차이가 있는지`에 대한 가설 검정을 시행하세요.

A : `38 33 35 92 76 97 88 41 11  9`

B : `18 52 62 48 30 40 87 12 97 82`

C :  `28 90  5 49 66 73 96 80  4 17`

D : ` 8 99  4 12  7 64 18 10  9 20`

